# **1. Corpus**

## Crawl URLs to extract all internal links 

**XENU Link Sleuth**  
https://home.snafu.de/tilman/xenulink.html 

*Le logiciel XENU Link Sleuth a finalement été retenu pour cette tâche* 

## Scrape textual data from crawled URLs
**BeautifulSoup HTML Parser**  
Réf : https://realpython.com/python-web-scraping-practical-introduction/

In [36]:
path = 'D:/1-XENU_crawl/csv_html_pdfs/'
acteur = "icm"

La liste des URLs à scrapper pour chaque corpus est contenue dans un fichier CSV. 
On commence donc par lire le CSV pour extraire nos URLS.

In [37]:
from pandas import *

regex = '.*png.*|.*jpeg.*|.*jpg.*|.*docx.*|.*js.*|.*font.*|.*gif.*|.*formulaire.*|.*?f%5B0%5D.*|.*img.*|.*%5Bfilter%.*|.*css.*|.*scripts.*|.*zip.*|.*xlsx.*|.*cms.*|.*pdf.*|.*/images/.*'


# encoding= 'ISO-8859-1' "utf-8"
path = 'D:/1-XENU_crawl/csv_html_pdfs/'
with open(path + acteur + '.csv', encoding='UTF-8') as f:
    csv = read_csv(f, sep=';')
    # Nettoyer ce qui ne devrait pas se trouver là
    csv = csv[~csv["Address"].str.contains(regex)] 

In [38]:
#liste = csv[csv['Type'] != 'application/pdf'] # On va scraper les PDFs avec une autre librairie que BeautifulSoup

liste = csv['Address'].tolist()
fr = csv[~csv["Address"].str.contains('/en/')]['Address'].tolist() # Données en français
en = csv[csv["Address"].str.contains('/en/')]['Address'].tolist() # Données en anglais
print("On va tenter d'aspirer {} pages Web".format(len(liste)))

On va tenter d'aspirer 1262 pages Web


In [39]:
import requests, re, ssl, os, sys, pandas as pd
from bs4 import BeautifulSoup
#from requests.packages.urllib3.util.retry import Retry

def getTextURL(url):    
    html = requests.get(url, headers = {'User-Agent': 'My User Agent 1.0'}, verify=False)
    html.encoding = 'utf-8'
    html = html.text
   
    soup = BeautifulSoup(html, "html.parser")
    data = soup.get_text(separator=' ').replace("\n", " ").replace("\r", " ").replace(u'\xa0', u' ')
    data = re.compile(r"\s+").sub(" ", data).strip() + "\n\n"
    
    return data

In [40]:
def scrape_list(x):
    output = []
    for site in liste: 
        try: 
            text = getTextURL(site)
            if not '���' in text:
                output.append({'url': site, 'text':text})
        except Exception as e:
            print("ERROR " + " - " + site)
            print(e)

    return output

In [41]:
sites_fr = scrape_list(fr)
if(len(en) > 0):
    sites_en = scrape_list(en)

c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.icm-mhi.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.icm-mhi.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.icm-mhi.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-

In [42]:
output_path = 'D:/1-XENU_crawl/scraping_07-2022/'

pd.DataFrame(sites_fr).to_csv(output_path + acteur + '.csv', escapechar='/')
if(len(en) > 0):
    pd.DataFrame(sites_en).to_csv(output_path + acteur + '_en.csv', escapechar='/')